# Phase 5: Latent Diffusion with VAE

In previous phases, we performed flow matching directly in **pixel space**. While this works well for small images (MNIST 28×28, CIFAR-10 32×32), it doesn't scale to high-resolution images.

## The Scaling Problem

Consider the computational cost of pixel-space diffusion:

| Resolution | Dimensions | Relative Cost |
|------------|------------|---------------|
| 32×32×3 | 3,072 | 1× |
| 64×64×3 | 12,288 | 4× |
| 256×256×3 | 196,608 | 64× |
| 512×512×3 | 786,432 | 256× |

Transformer self-attention has O(N²) complexity in sequence length. For images:
- 32×32 with 4×4 patches = 64 tokens → 4,096 attention pairs
- 256×256 with 4×4 patches = 4,096 tokens → 16,777,216 attention pairs

This is **4,096× more computation** just for attention!

## The Solution: Latent Diffusion

The key insight of **Stable Diffusion** (Rombach et al., 2022):

> Instead of diffusing in pixel space, first compress images to a smaller latent space, then do diffusion there.

```
Pixel Space Diffusion:
  noise (256×256×3) ──flow matching──> image (256×256×3)
  
Latent Space Diffusion:
  noise (32×32×4) ──flow matching──> latent (32×32×4) ──VAE decode──> image (256×256×3)
```

With 8× spatial compression:
- 256×256×3 image → 32×32×4 latent
- 196,608 → 4,096 dimensions (48× smaller!)

## What You'll Learn

1. **Variational Autoencoders (VAEs)**: How to compress images to latent space
2. **The Reparameterization Trick**: How to backpropagate through sampling
3. **Latent Flow Matching**: Training diffusion in compressed space
4. **The Full Pipeline**: Encode → Denoise → Decode

## Mathematical Framework

### VAE: Learning a Latent Space

A VAE learns two functions:

**Encoder**: $q_\phi(z|x) = \mathcal{N}(\mu_\phi(x), \sigma^2_\phi(x))$
- Maps image $x$ to a distribution over latents $z$
- Outputs mean $\mu$ and variance $\sigma^2$ of a Gaussian

**Decoder**: $p_\theta(x|z)$
- Maps latent $z$ back to image $x$
- Learns to reconstruct the original image

### Training Objective (ELBO)

The VAE is trained to maximize the Evidence Lower Bound:

$$\log p(x) \geq \mathbb{E}_{q(z|x)}[\log p(x|z)] - D_{KL}(q(z|x) \| p(z))$$

In practice, we minimize:

$$\mathcal{L}_{VAE} = \underbrace{\|x - \text{decode}(\text{encode}(x))\|^2}_{\text{reconstruction}} + \beta \cdot \underbrace{D_{KL}(q(z|x) \| \mathcal{N}(0,I))}_{\text{regularization}}$$

### Latent Diffusion

Once the VAE is trained, we do flow matching in latent space:

1. **Encode training data**: $z_0 = \text{encode}(x)$
2. **Flow matching on latents**: Learn $v_\theta(z_t, t)$ where $z_t = (1-t)z_0 + tz_1$
3. **Generate**: Sample $z_1 \sim \mathcal{N}(0,I)$, integrate to $z_0$, decode to $x = \text{decode}(z_0)$

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

# Auto-reload modules during development
%load_ext autoreload
%autoreload 2

# Set up device
from text_to_image import get_device
device = get_device()
print(f"Using device: {device}")

## 1. Variational Autoencoders: The Compression Engine

### Why VAE Instead of Regular Autoencoder?

A regular autoencoder learns a deterministic mapping:
$$x \xrightarrow{\text{encode}} z \xrightarrow{\text{decode}} \hat{x}$$

The problem: the latent space may have "holes" - regions where no training data maps to, producing garbage when decoded.

A VAE learns a **probabilistic** mapping:
$$x \xrightarrow{\text{encode}} (\mu, \sigma^2) \xrightarrow{\text{sample}} z \xrightarrow{\text{decode}} \hat{x}$$

The KL regularization term encourages the latent distribution to be close to $\mathcal{N}(0, I)$, ensuring:
1. **Smooth latent space**: Nearby points decode to similar images
2. **No holes**: The entire space is "covered" by the prior
3. **Sampling works**: We can sample $z \sim \mathcal{N}(0, I)$ and decode to valid images

### The Reparameterization Trick

The encoder outputs $\mu$ and $\log\sigma^2$ (log-variance for numerical stability).

To sample $z \sim \mathcal{N}(\mu, \sigma^2)$ while allowing backpropagation:

$$z = \mu + \sigma \cdot \epsilon, \quad \epsilon \sim \mathcal{N}(0, I)$$

This moves the randomness ($\epsilon$) outside the computational graph, making $z$ differentiable with respect to $\mu$ and $\sigma$.

### Architecture Overview

```
Image (32×32×3)                                    Image (32×32×3)
      |                                                  ^
      v                                                  |
+------------+                                    +------------+
|  Encoder   |  (downsample 4×)                   |  Decoder   |  (upsample 4×)
+------------+                                    +------------+
      |                                                  ^
      v                                                  |
  (μ, log σ²)  ───> sample z = μ + σε ───────────>    z
   (8×8×8)              (8×8×4)                     (8×8×4)
```

Note: The encoder outputs 2× the latent channels (for $\mu$ and $\log\sigma^2$), then we sample to get the final latent.

In [ ]:
# Load CIFAR-10 for this demonstration
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # [-1, 1]
])

train_dataset = torchvision.datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=transform
)

train_loader = DataLoader(
    train_dataset,
    batch_size=128,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

print(f"Dataset size: {len(train_dataset):,} images")
print(f"Image shape: {train_dataset[0][0].shape}")
print(f"Pixel range: [{train_dataset[0][0].min():.1f}, {train_dataset[0][0].max():.1f}]")

In [ ]:
# Visualize some training samples
def show_images(images, nrow=8, title=""):
    """Display a grid of images."""
    images = (images + 1) / 2  # Denormalize from [-1, 1] to [0, 1]
    images = images.clamp(0, 1)
    
    grid = torchvision.utils.make_grid(images, nrow=nrow, padding=2)
    plt.figure(figsize=(12, 12 * grid.shape[1] / grid.shape[2]))
    plt.imshow(grid.permute(1, 2, 0).cpu().numpy())
    plt.axis('off')
    if title:
        plt.title(title, fontsize=14)
    plt.show()

# Get a batch and visualize
sample_batch, sample_labels = next(iter(train_loader))
show_images(sample_batch[:32], title="CIFAR-10 Training Samples")

## 2. Building the VAE

Our VAE architecture:

**Encoder** (32×32 → 8×8, 4× spatial compression):
- Input conv: 3 → 64 channels
- Downsample block 1: 64 → 64, stride 2 (32→16)
- Downsample block 2: 64 → 128, stride 2 (16→8)
- Output conv: 128 → 8 (4 for μ, 4 for log σ²)

**Decoder** (8×8 → 32×32):
- Input conv: 4 → 128 channels
- Upsample block 1: 128 → 64, upsample 2× (8→16)
- Upsample block 2: 64 → 64, upsample 2× (16→32)
- Output conv: 64 → 3

**Compression Ratio**:
- Pixel space: 32×32×3 = 3,072 values
- Latent space: 8×8×4 = 256 values
- **12× compression**

In [ ]:
from text_to_image.vae import SmallVAE

# Create VAE for CIFAR-10 (32×32 RGB)
vae = SmallVAE(
    in_channels=3,        # RGB
    latent_channels=4,    # 4 latent channels
    hidden_channels=64,   # Base hidden channels
).to(device)

num_params = sum(p.numel() for p in vae.parameters() if p.requires_grad)
print(f"VAE parameters: {num_params:,}")

# Test forward pass
test_img = torch.randn(4, 3, 32, 32, device=device)
with torch.no_grad():
    recon, mean, logvar = vae(test_img)
    latent = vae.encode(test_img)

print(f"\nShape analysis:")
print(f"  Input image:  {test_img.shape}  ({test_img.numel() // 4:,} values per image)")
print(f"  Latent mean:  {mean.shape}  ({mean.numel() // 4:,} values per image)")
print(f"  Latent z:     {latent.shape}")
print(f"  Recon image:  {recon.shape}")
print(f"\nCompression ratio: {test_img.numel() / latent.numel():.1f}×")

## 3. The VAE Loss Function

### Reconstruction Loss

Simple MSE between input and reconstruction:

$$\mathcal{L}_{recon} = \|x - \hat{x}\|^2 = \|x - \text{decode}(\text{encode}(x))\|^2$$

### KL Divergence Loss

The KL divergence between the encoder distribution and the prior:

$$D_{KL}(q(z|x) \| p(z)) = D_{KL}(\mathcal{N}(\mu, \sigma^2) \| \mathcal{N}(0, I))$$

For Gaussians, this has a closed form:

$$D_{KL} = -\frac{1}{2} \sum_{i=1}^{d} \left(1 + \log\sigma_i^2 - \mu_i^2 - \sigma_i^2\right)$$

**Intuition**:
- If $\mu \to 0$ and $\sigma \to 1$: KL → 0 (matches prior)
- If $\mu$ is large: KL increases (penalizes mean far from 0)
- If $\sigma$ is very small or large: KL increases (penalizes variance different from 1)

### Combining the Losses

$$\mathcal{L}_{VAE} = \mathcal{L}_{recon} + \beta \cdot D_{KL}$$

For **latent diffusion**, we use very small $\beta$ (e.g., 0.00001) because:
1. We prioritize reconstruction quality (the decoder needs to produce sharp images)
2. The diffusion model will handle generation (so we don't need perfect sampling from the prior)
3. We just need good encoding/decoding

In [ ]:
# Visualize KL divergence behavior
def kl_divergence(mu, logvar):
    """KL divergence from N(mu, sigma^2) to N(0, 1)."""
    return -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

# Test different scenarios
scenarios = [
    ("μ=0, σ=1 (perfect match)", 0.0, 0.0),
    ("μ=1, σ=1 (shifted mean)", 1.0, 0.0),
    ("μ=0, σ=0.5 (narrow)", 0.0, np.log(0.25)),
    ("μ=0, σ=2 (wide)", 0.0, np.log(4.0)),
    ("μ=2, σ=0.5 (shifted+narrow)", 2.0, np.log(0.25)),
]

print("KL Divergence Examples (per dimension):")
print("-" * 50)
for name, mu_val, logvar_val in scenarios:
    mu = torch.tensor([mu_val])
    logvar = torch.tensor([logvar_val])
    kl = kl_divergence(mu, logvar).item()
    print(f"{name:35s}: KL = {kl:.4f}")

print("\nThe KL term encourages μ→0 and σ→1 (standard normal).")

## 4. Training the VAE

### Training Algorithm

For each batch:
1. Forward pass: $x \to (\mu, \log\sigma^2) \to z \to \hat{x}$
2. Compute reconstruction loss: $\mathcal{L}_{recon} = \|x - \hat{x}\|^2$
3. Compute KL loss: $\mathcal{L}_{KL} = D_{KL}(q(z|x) \| p(z))$
4. Total loss: $\mathcal{L} = \mathcal{L}_{recon} + \beta \cdot \mathcal{L}_{KL}$
5. Backpropagate and update

### $\beta$ Schedule

For latent diffusion, we use $\beta \approx 10^{-5}$. This extremely small value means:
- Reconstruction quality is prioritized
- Latents may not perfectly follow $\mathcal{N}(0,I)$
- But that's OK - we'll compute a scale factor to normalize them

In [ ]:
from text_to_image.train import VAETrainer

# Create trainer
vae_trainer = VAETrainer(
    model=vae,
    dataloader=train_loader,
    lr=1e-4,
    weight_decay=0.01,
    kl_weight=0.00001,  # Very small β for latent diffusion
    device=device,
)

# Train VAE
VAE_EPOCHS = 30  # Increase for better reconstruction
print(f"Training VAE for {VAE_EPOCHS} epochs...")
print(f"KL weight (β): {vae_trainer.kl_weight}")
print()

vae_losses = vae_trainer.train(num_epochs=VAE_EPOCHS)

In [ ]:
# Plot VAE training loss
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(vae_trainer.losses, label='Total Loss')
axes[0].plot(vae_trainer.recon_losses, label='Reconstruction Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('VAE Training Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(vae_trainer.kl_losses)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('KL Divergence')
axes[1].set_title('KL Divergence over Training')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Final reconstruction loss: {vae_trainer.recon_losses[-1]:.4f}")
print(f"Final KL divergence: {vae_trainer.kl_losses[-1]:.4f}")

In [ ]:
# Visualize VAE reconstructions
vae.eval()

test_batch = sample_batch[:16].to(device)

with torch.no_grad():
    recon, _, _ = vae(test_batch)

# Show original vs reconstruction
fig, axes = plt.subplots(2, 1, figsize=(14, 7))

# Original images
original_grid = torchvision.utils.make_grid((test_batch + 1) / 2, nrow=16, padding=2)
axes[0].imshow(original_grid.permute(1, 2, 0).cpu().numpy())
axes[0].set_title('Original Images', fontsize=12)
axes[0].axis('off')

# Reconstructed images
recon_grid = torchvision.utils.make_grid((recon + 1) / 2, nrow=16, padding=2)
axes[1].imshow(recon_grid.permute(1, 2, 0).cpu().clamp(0, 1).numpy())
axes[1].set_title('VAE Reconstructions', fontsize=12)
axes[1].axis('off')

plt.tight_layout()
plt.show()

# Compute reconstruction error
mse = F.mse_loss(recon, test_batch).item()
print(f"\nReconstruction MSE: {mse:.4f}")
print("The VAE successfully compresses and reconstructs images!")

## 5. Exploring the Latent Space

Let's visualize what the latent space looks like.

### Latent Statistics

After training, we compute a **scale factor** to normalize the latent space:

$$z_{normalized} = z / \sigma_{data}$$

where $\sigma_{data}$ is the empirical standard deviation of encoded latents.

This ensures the latent space has approximately unit variance, matching the noise distribution $\mathcal{N}(0, I)$ we'll use for flow matching.

In [ ]:
# Analyze latent space statistics
vae.eval()

all_means = []
all_stds = []

with torch.no_grad():
    for i, (images, _) in enumerate(train_loader):
        if i >= 50:  # Use 50 batches for stats
            break
        images = images.to(device)
        z = vae.encode(images, sample=False)  # Use mean encoding
        all_means.append(z.mean().item())
        all_stds.append(z.std().item())

avg_mean = np.mean(all_means)
avg_std = np.mean(all_stds)

print(f"Latent Space Statistics:")
print(f"  Mean: {avg_mean:.4f} (ideally ≈ 0)")
print(f"  Std:  {avg_std:.4f} (will be normalized to ≈ 1)")
print(f"\nVAE scale factor: {vae.scale_factor.item():.4f}")

In [ ]:
# Visualize latent channels
test_img = sample_batch[0:1].to(device)

with torch.no_grad():
    z = vae.encode(test_img, sample=False)

fig, axes = plt.subplots(1, 5, figsize=(15, 3))

# Original image
axes[0].imshow((test_img[0].cpu().permute(1, 2, 0) + 1) / 2)
axes[0].set_title('Original\n(32×32×3)')
axes[0].axis('off')

# Each latent channel
for i in range(4):
    latent_ch = z[0, i].cpu().numpy()
    axes[i+1].imshow(latent_ch, cmap='RdBu', vmin=-2, vmax=2)
    axes[i+1].set_title(f'Latent Ch {i}\n(8×8)')
    axes[i+1].axis('off')

plt.suptitle('Image to Latent: 32×32×3 → 8×8×4 (12× compression)', fontsize=12)
plt.tight_layout()
plt.show()

print(f"\nLatent shape: {z.shape}")
print(f"Latent values: min={z.min():.2f}, max={z.max():.2f}")

## 6. Flow Matching in Latent Space

Now we train a DiT to perform flow matching **in the latent space** instead of pixel space.

### The Key Insight

Everything is the same as pixel-space flow matching, but we work with latents:

| Pixel Space | Latent Space |
|-------------|---------------|
| $x_0$ = image | $z_0$ = encode(image) |
| $x_1 \sim \mathcal{N}(0, I)$ | $z_1 \sim \mathcal{N}(0, I)$ |
| $x_t = (1-t)x_0 + tx_1$ | $z_t = (1-t)z_0 + tz_1$ |
| $v = x_1 - x_0$ | $v = z_1 - z_0$ |
| Generate $x$ directly | Generate $z$, then decode |

### DiT Architecture for Latent Space

The DiT operates on the latent shape (8×8×4 for our VAE):
- `in_channels=4` (latent channels)
- `img_size=8` (latent spatial size)
- `patch_size=2` (4×4 = 16 patches)

This is **much smaller** than pixel-space DiT!

In [ ]:
from text_to_image.dit import DiT

# Create DiT for latent space
# Latent shape: (4, 8, 8)
latent_dit = DiT(
    img_size=8,           # Latent spatial size
    patch_size=2,         # 2×2 patches → 4×4 = 16 tokens
    in_channels=4,        # Latent channels
    embed_dim=256,        # Transformer embedding dimension
    depth=6,              # Number of transformer blocks
    num_heads=8,          # Attention heads
    mlp_ratio=4.0,
).to(device)

latent_dit_params = sum(p.numel() for p in latent_dit.parameters())

# Compare with pixel-space DiT
pixel_dit = DiT(
    img_size=32,
    patch_size=4,
    in_channels=3,
    embed_dim=256,
    depth=6,
    num_heads=8,
)
pixel_dit_params = sum(p.numel() for p in pixel_dit.parameters())

print("Model Comparison:")
print(f"  Pixel-space DiT: {pixel_dit_params:,} params, {8*8}=64 tokens")
print(f"  Latent-space DiT: {latent_dit_params:,} params, {4*4}=16 tokens")
print(f"\nToken reduction: {64/16:.0f}×")
print(f"Attention cost reduction: {(64*64)/(16*16):.0f}×")

In [ ]:
from text_to_image.train import LatentDiffusionTrainer

# Create latent diffusion trainer
latent_trainer = LatentDiffusionTrainer(
    model=latent_dit,
    vae=vae,
    dataloader=train_loader,
    lr=1e-4,
    weight_decay=0.01,
    device=device,
)

# Train
LATENT_EPOCHS = 30  # Increase for better results
print(f"Training Latent Diffusion for {LATENT_EPOCHS} epochs...")
print(f"VAE is frozen - only DiT is trained.")
print()

latent_losses = latent_trainer.train(num_epochs=LATENT_EPOCHS)

In [ ]:
# Plot training loss
plt.figure(figsize=(10, 4))
plt.plot(latent_losses)
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('Latent Diffusion Training Loss')
plt.grid(True, alpha=0.3)
plt.show()

print(f"Final loss: {latent_losses[-1]:.4f}")

## 7. Sampling from Latent Space

Generation in latent diffusion:

1. **Sample noise in latent space**: $z_1 \sim \mathcal{N}(0, I)$, shape (4, 8, 8)
2. **Integrate ODE in latent space**: $z_1 \to z_0$ using learned velocity
3. **Decode to pixels**: $x = \text{decode}(z_0)$, shape (3, 32, 32)

### Why This Is Efficient

All the expensive ODE integration happens in the small latent space:
- 50 steps × 16 tokens = 800 attention operations per image
- vs. 50 steps × 64 tokens = 3,200 operations in pixel space

The VAE decode is just one forward pass at the end.

In [ ]:
from text_to_image.sampling import sample_latent

# Generate samples
latent_dit.eval()
vae.eval()

print("Generating samples via latent diffusion...")
print("  1. Sample noise in latent space (4×8×8)")
print("  2. Integrate ODE from t=1 to t=0")
print("  3. Decode to pixel space (3×32×32)")
print()

with torch.no_grad():
    generated = sample_latent(
        model=latent_dit,
        vae=vae,
        num_samples=32,
        latent_shape=(4, 8, 8),
        num_steps=50,
        device=device,
    )

show_images(generated, nrow=8, title="Generated Samples (Latent Diffusion)")

In [ ]:
# Visualize the generation process in latent space
with torch.no_grad():
    generated, trajectory = sample_latent(
        model=latent_dit,
        vae=vae,
        num_samples=4,
        latent_shape=(4, 8, 8),
        num_steps=50,
        device=device,
        return_trajectory=True,
    )

# Show evolution of one sample
steps_to_show = [0, 10, 20, 30, 40, 50]
fig, axes = plt.subplots(2, len(steps_to_show), figsize=(15, 6))

for col, step_idx in enumerate(steps_to_show):
    t_val = 1.0 - step_idx / 50
    
    # Latent (show first channel)
    latent = trajectory[step_idx][0, 0].cpu().numpy()
    axes[0, col].imshow(latent, cmap='RdBu', vmin=-3, vmax=3)
    axes[0, col].set_title(f't={t_val:.2f}')
    axes[0, col].axis('off')
    
    # Decoded image
    with torch.no_grad():
        decoded = vae.decode(trajectory[step_idx][:1])
    img = (decoded[0].cpu().permute(1, 2, 0) + 1) / 2
    axes[1, col].imshow(img.clamp(0, 1).numpy())
    axes[1, col].axis('off')

axes[0, 0].set_ylabel('Latent\n(8×8)', fontsize=10)
axes[1, 0].set_ylabel('Decoded\n(32×32)', fontsize=10)

plt.suptitle('Latent Diffusion: ODE Integration + VAE Decode', fontsize=12)
plt.tight_layout()
plt.show()

print("\nObserve how:")
print("  - t=1: Pure noise in latent space → noisy decoded image")
print("  - t→0: Structure emerges in latent → coherent image forms")

## 8. Class-Conditional Latent Diffusion

We can add class conditioning just like in Phase 3, but in latent space.

The architecture is identical to `ConditionalDiT`, just operating on latent shapes.

In [ ]:
from text_to_image.dit import ConditionalDiT

# Create class-conditional DiT for latent space
cond_latent_dit = ConditionalDiT(
    num_classes=10,       # CIFAR-10 classes
    img_size=8,           # Latent spatial size
    patch_size=2,         # 2×2 patches
    in_channels=4,        # Latent channels
    embed_dim=256,
    depth=6,
    num_heads=8,
    mlp_ratio=4.0,
).to(device)

print(f"Conditional Latent DiT parameters: {sum(p.numel() for p in cond_latent_dit.parameters()):,}")

In [ ]:
from text_to_image.train import LatentConditionalTrainer

# Train conditional latent diffusion
cond_latent_trainer = LatentConditionalTrainer(
    model=cond_latent_dit,
    vae=vae,
    dataloader=train_loader,
    lr=1e-4,
    weight_decay=0.01,
    label_drop_prob=0.1,  # 10% dropout for CFG
    num_classes=10,
    device=device,
)

COND_EPOCHS = 30
print(f"Training Conditional Latent Diffusion for {COND_EPOCHS} epochs...")
print()

cond_losses = cond_latent_trainer.train(num_epochs=COND_EPOCHS)

In [ ]:
from text_to_image.sampling import sample_latent_conditional

# CIFAR-10 class names
CIFAR10_CLASSES = [
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'
]

cond_latent_dit.eval()

# Generate one sample per class
print("Generating one sample per class with CFG...")

with torch.no_grad():
    class_samples = sample_latent_conditional(
        model=cond_latent_dit,
        vae=vae,
        class_labels=list(range(10)),
        latent_shape=(4, 8, 8),
        num_steps=50,
        cfg_scale=3.0,
        device=device,
        num_classes=10,
    )

# Display with class labels
fig, axes = plt.subplots(2, 5, figsize=(15, 6))

for i, (ax, class_name) in enumerate(zip(axes.flat, CIFAR10_CLASSES)):
    img = (class_samples[i].cpu().permute(1, 2, 0) + 1) / 2
    ax.imshow(img.clamp(0, 1).numpy())
    ax.set_title(class_name)
    ax.axis('off')

plt.suptitle('Class-Conditional Latent Diffusion (CFG=3.0)', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Compare CFG scales
cfg_scales = [1.0, 2.0, 3.0, 5.0]
target_class = 3  # cat

fig, axes = plt.subplots(1, len(cfg_scales), figsize=(16, 4))

for ax, scale in zip(axes, cfg_scales):
    torch.manual_seed(42)  # Same seed for comparison
    
    with torch.no_grad():
        sample = sample_latent_conditional(
            model=cond_latent_dit,
            vae=vae,
            class_labels=[target_class],
            latent_shape=(4, 8, 8),
            num_steps=50,
            cfg_scale=scale,
            device=device,
        )
    
    img = (sample[0].cpu().permute(1, 2, 0) + 1) / 2
    ax.imshow(img.clamp(0, 1).numpy())
    ax.set_title(f'CFG={scale}')
    ax.axis('off')

plt.suptitle(f'Effect of CFG Scale (class: {CIFAR10_CLASSES[target_class]})', fontsize=12)
plt.tight_layout()
plt.show()

print("Higher CFG → stronger class adherence but potentially less diversity")

## 9. Computational Comparison

Let's compare the computational requirements of pixel-space vs latent-space diffusion.

In [ ]:
import time

# Time comparison
num_samples = 16
num_steps = 50

# Latent space timing
torch.cuda.synchronize() if torch.cuda.is_available() else None
start = time.time()

with torch.no_grad():
    _ = sample_latent(
        model=latent_dit,
        vae=vae,
        num_samples=num_samples,
        latent_shape=(4, 8, 8),
        num_steps=num_steps,
        device=device,
    )

torch.cuda.synchronize() if torch.cuda.is_available() else None
latent_time = time.time() - start

print(f"Latent Diffusion ({num_samples} samples, {num_steps} steps):")
print(f"  Time: {latent_time:.2f}s")
print(f"  Per image: {latent_time/num_samples*1000:.1f}ms")
print()
print("Latent space details:")
print(f"  Latent shape: (4, 8, 8) = 256 values")
print(f"  Patches: 4×4 = 16 tokens")
print(f"  Attention per step: 16×16 = 256 pairs")

## Summary: The Complete Latent Diffusion Pipeline

### Architecture

```
TRAINING:
                                                      Flow Matching Loss
                                                             ↑
Image x ───────> [VAE Encoder] ───> z_0 ───> z_t ───> [DiT] ───> v_pred
                   (frozen)          ↑         ↑
                                     |    noise z_1, time t
                                     |
                              scale_factor

INFERENCE:
Noise z_1 ───> [DiT + ODE] ───> z_0 ───> [VAE Decoder] ───> Image x
```

### Key Equations

| Component | Equation | Purpose |
|-----------|----------|----------|
| VAE Encode | $z = \mu + \sigma \epsilon$ | Compress to latent |
| VAE Decode | $\hat{x} = D(z)$ | Reconstruct image |
| VAE Loss | $\mathcal{L} = \|x-\hat{x}\|^2 + \beta D_{KL}$ | Train VAE |
| Latent Interpolation | $z_t = (1-t)z_0 + tz_1$ | Flow path |
| Latent Velocity | $v = z_1 - z_0$ | Target |
| Training Loss | $\mathcal{L} = \|v_\theta(z_t, t) - v\|^2$ | Train DiT |
| Generation | $z_0 = z_1 - \int_1^0 v_\theta(z_t, t) dt$ | ODE solve |

### Compression Ratios

| Resolution | Pixel Dims | Latent Dims | Compression |
|------------|------------|-------------|-------------|
| 32×32×3 | 3,072 | 8×8×4 = 256 | 12× |
| 64×64×3 | 12,288 | 8×8×4 = 256 | 48× |
| 256×256×3 | 196,608 | 32×32×4 = 4,096 | 48× |
| 512×512×3 | 786,432 | 64×64×4 = 16,384 | 48× |

### Advantages of Latent Diffusion

1. **Efficiency**: 48× fewer dimensions means faster training and inference
2. **Scalability**: Can handle high-resolution images that would be infeasible in pixel space
3. **Quality**: VAE provides perceptually meaningful compression
4. **Flexibility**: Same DiT architecture works across resolutions

### This Is How Stable Diffusion Works!

The techniques in this notebook are the foundation of Stable Diffusion:
- VAE compresses 512×512 images to 64×64 latents
- U-Net (or DiT) does flow matching in latent space
- CLIP text encoder provides conditioning
- CFG provides strong text adherence

Congratulations! You now understand the complete text-to-image pipeline.

In [ ]:
# Save trained models
vae_trainer.save_checkpoint("phase5_vae.pt")
latent_trainer.save_checkpoint("phase5_latent_dit.pt")
cond_latent_trainer.save_checkpoint("phase5_cond_latent_dit.pt")

print("Models saved!")
print("  - phase5_vae.pt")
print("  - phase5_latent_dit.pt")
print("  - phase5_cond_latent_dit.pt")